# PIC-SURE API Use-Case: Querying on Genomic Variables

This tutorial notebook focuses how to use the PIC-SURE API to query genomic and phenotypic data together.

For a more basic introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.
 
**Before running this notebook, please be sure to get a user-specific security token. For more information about how to proceed, see the \"Get your security token\" instructions in the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token).**

 -------

# Environment set-up

### Pre-requisites
- R 3.4 or later

### Install packages

**Note that if you are using the dedicated PIC-SURE environment within the BioData Catalyst Seven Bridges platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in Seven Bridges, or if you do not have all the necessary dependencies installed.
#install.packages(c("devtools", "dplyr"))

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="ALS-3799", force=T, quiet=FALSE)
library(dplyr)

### Connecting to a PIC-SURE resource

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")
     

# Example 1: Walkthrough on building a query with genomic and phenotypic varibles
## 1.1 Building the query with the PIC-SURE API

We are going to create a new query request from the PIC-SURE resource that was specified above.  For this example, we will limit the query to a single study, a single phenotype (gender and age range), and two genomic filters. 

First we will create a new query instance.

In [ ]:
my_query <- picsure::bdc.newQuery(session) # Initialize a new query


### Find all SAGE variables
We can search for variables related to our search query using the `bdc::find.in.dictionary` function. 

In this example, we will retrive all variables available in the SAGE study (NHLBI TOPMed: Study of African Americans, Asthma, Genes and Environment (SAGE) Study , phs000921).

You can find information about the phs number associated with each study and what data are available from the Data Access Dashboard in the PIC-SURE [User Interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/).

In [ ]:
sage_df <- picsure::bdc.searchPicsure(session, 'phs000921', includeValues = TRUE)  


## 1.2 Add categorical phenotypic variable (sex) to the query

First, we will search our sage data dictionary for a sex-related variable.

In [ ]:
sex_var <- sage_df %>% filter(grepl('sex', var_name, ignore.case = TRUE))
sex_var

Let's examine the values within this variable.

In [ ]:
sex_var %>% pull(values)

We are only interested in FEMALE values in this use case - let's apply this filter to our query accordingly.

In [ ]:
my_query <- picsure::addClause(query = my_query, 
                               keys = sex_var %>% pull(name),
                               type = 'FILTER',
                               categories = list('FEMALE'))

## 1.3 Add continuous phenotypic variable (age) to the query
For this example, we are only interested in children (<=18 years of age).

Following the data dictionary search pattern shown above, we can search for the SAGE study variables related to age.

In [ ]:
# Search our SAGE dataframe for an age variable
age_var <- sage_df %>% filter(grepl('age', var_name, ignore.case = TRUE))
print(paste0(age_var$var_name, ': ', age_var$name))

# Examine the values
print(paste0('Min: ', age_var$min, ', Max: ', age_var$max))

# Filter to children aged 18 years or younger
my_query <- picsure::addClause(query = my_query, 
                               keys = age_var$name,
                               type = 'FILTER',
                               min = 7.3, max = 18)

## 1.4 Add genomic filters to the query
To start adding genomic filters to our query, we first need to understand what genomic annotations are available.

In [ ]:
genotype_annotations <- picsure::getGenomicAnnotations(session)
genotype_annotations[2:5,]

As shown in the genomic_annotations object above, some genomic variables that can be used in queries include `Gene_with_variant`, `Variant_class`, and `Variant_severity`.

#### Add genotypic variable (Gene_with_variant) to the query

Let's explore what genes we can filter our variants by by looking at the values associated with the `Gene_with_variant` annotation.

In [ ]:
# get total list of genes
genes <- genotype_annotations %>% filter(name == 'Gene_with_variant') %>% pull(values)
genes <- strsplit(genes, ', ')[[1]]

# print the first 10 genes
print(genes[1:10])

# check if a certain gene of interest, e.g. CHD8, is in the gene list
gene_of_interest <- 'CHD8'
gene_of_interest %in% genes

We can view the full list of `Variant_consequence_calculated` options in a similar way.

In [ ]:
# get all "Variant_consequence_calculated" values
consequences = genotype_annotations %>% filter(name == 'Variant_consequence_calculated') %>% pull(values)
consequences <- strsplit(consequences, ', ')[[1]]
print(consequences)

The gene list shown above provides a list of values that can be used for the `Gene_with_variant`, in this case genes affected by a variant. Let's say we are interested in participants who have some kind of variant on the CHD8 gene. We can add this genomic filter to our query like so:

In [ ]:
my_query <- picsure::addClause(query = my_query, 
                               keys = 'Gene_with_variant',
                               type = 'FILTER',
                               categories = list('CHD8'))

We can further narrow our query by filtering on variant consequence. Let's look at missense variants in this example. 

In [ ]:
my_query <- picsure::addClause(query = my_query, 
                               keys = 'Variant_consequence_calculated',
                               type = 'FILTER',
                               categories = list('missense_variant'))

## 1.5 Retrieve data from the query

#### Getting query count
We have now built a query called `my_query` which contains the search criteria we are interested in:
- Sex = Female
- Age < 18
- Participants have a missense variant on CHD8

We will run a count query to find the number of matching participants.
This is a great way to preview how many participants match your query criteria without extracting all of the data yet.


In [ ]:
picsure::runQuery(my_query, resultType = 'Count')

#### Getting query data
We will retrieve our results in a dataframe.
Note that since we only added sex and age to the query, these are the only phenotypic variables returned.

In [ ]:
query_result <- picsure::runQuery(my_query)

In [ ]:
dim(query_result)

In [ ]:
head(query_result)

# Example 2: Use case with *SERPINA1* gene variants as a risk factor for  COPD

In this example, we will create a query to explore the relationship between the COPD phenotype and variants in the SERPINA1 gene. Variations of the SERPINA1 gene have been found to be a strong risk factor for COPD, which you can read more about [here](https://pubmed.ncbi.nlm.nih.gov/31661293/).

To explore this relationship, we will narrow the cohort down to participants that meet the following criteria:
* participated in the COPDgene study
* have had COPD
* have a *SERPINA1* gene variant with high or moderate severity

## 2.1 Create query
Let's start by creating a new query and finding the variables pertaining to the COPDgene study (phs000179)

In [ ]:
# Retrieve all COPDGene variables
copd_df <- picsure::bdc.searchPicsure(session, 'phs000179', includeValues = TRUE)  


### Criteria 1: Participants who have had COPD
Let's search our dataframe of COPDGene variables to find the one we want to use to filter for participants who have ever had COPD.

In [ ]:
copd_df %>% 
    filter(grepl('copd', var_description, ignore.case = TRUE)) %>%
    select(name, var_name, var_description, values)

The variable with the description "COPD: have you ever had COPD" seems the most promising. Let's copy that variable's HPDS path to use in our query. Also note what values are present for this variable- we will filter to participants with value "Yes".

In [ ]:
copd_path <- '\\phs000179\\pht002239\\phv00159731\\COPD\\'

In [ ]:
copd_query <- picsure::bdc.newQuery(session) # Initialize a new query

copd_query <- picsure::addClause(query = copd_query, 
                                 keys = copd_path,
                                 type = 'FILTER',
                                 categories = list('Yes'))


### Criteria 2: Participants with variants of high or moderate severity on *SERPINA1*

In [ ]:
copd_query <- picsure::addClause(query = copd_query, 
                                 keys = 'Gene_with_variant',
                                 type = 'FILTER',
                                 categories = list('SERPINA1'))

In [ ]:
# what values are available for Variant_severity?
genotype_annotations %>% filter(name == "Variant_severity") %>% pull(values)

In [ ]:
copd_query <- picsure::addClause(query = copd_query, 
                                 keys = 'Variant_severity',
                                 type = 'FILTER',
                                 categories = list("HIGH", "MODERATE"))

### 2.2 Get Results
Now that the filtering is complete, we can use this final query to get counts and perform analysis on the data.

In [ ]:
picsure::runQuery(copd_query, resultType = 'count')

In [ ]:
copd_result <- picsure::runQuery(copd_query)

In [ ]:
dim(copd_result)

In [ ]:
head(copd_result)